# Logging in Jupyter notebooks

When evolving your notebook code, as it becomes more complex, your logging attempts will be forced to keepup: usually taking this path.

 - add `print` statements in your cells
 - tire of constant commenting/uncommenting the `prints`
 - research logging and learn that you can use python's loggers in notebooks! Yay!
 - research further and figure out that you can direct builting loggers for your imports as well.
 - Look at this verbose output 😖!! Discover the power of visual separation with colors! 

This notebook demonstrates the above evolution. 

## Basic Logging

The cell below shows how you'd setup basic logging. The `reload(logging)` is needed because of the singleton aspect of the config. In a jupyter environment _(and you might face this in your standalone scripts as well)_, turns out the something in the infrastructure already initializes the logger. Once that is done, subsequent calls to `logging.basicConfig` do not have any effect. This is where `reload` comes in as it clears the module state allowing the next `logging.basicConfig` to take effect.

> This is also the reason why you will need to restart the notebook kernel if you want to change your `logging.basicConfig` args. Switch from `logging.DEBUG` to `logging.INFO` say.

In [3]:
# Setup logging 
# Note that module needs to be reloaded for our config to take as Jupyter already configures it
# which makes all future configs no-ops.
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', 
                    level=logging.DEBUG, 
                    datefmt='%I:%M:%S')

and this is how you would introduce loggging into your code.

In [4]:
def my_func():
    # do stuff and then
    logging.debug("My debug statement")
    logging.warning("My warning")
    logging.error("My error")

my_func()

01:54:00 DEBUG:My debug statement
01:54:00 WARNING:My warning
01:54:00 ERROR:My error


👆 should show the three log statements you just printed out from the cell above.

You can setup a variable in a top-most cell `MY_LOG = logging.WARNING` and use that in `basicConfig(.. level=MY_LOG)` or directly change the `logging.basicConfig` statement to set it for a specific notebook run. Play with adjusting the `level` argument and notice how the output gets filtered.

## Configure for colab environment

In [5]:
import os

# You could do one of the two.
# Either paste your OpenAI Key here or put it in secrets
if 'google.colab' in str(get_ipython()):
  from google.colab import userdata
  logging.debug("Tryign to fetch OPENAI_API_KEY from your secrets. Remember to make it available to this notebook")
  os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

logging.debug("Checking if OPENAI_API_KEY is available")
assert(os.environ.get("OPENAI_API_KEY"))

01:54:03 DEBUG:Checking if OPENAI_API_KEY is available



## Control logging from imports - OpenAI - Env vars

With LLMs, for instance, there are many times when you want visibility into low level decision making. Particularly those that might cause latency spikes, like HTTP response codes, hitting rate-limits and automatic retries. Many of these log statements will also turn out to be discovery and make you research the API in more detail as you suddenly notice something that might be relevant to control or atleast surface.

This section will demonstrate controlling `OpenAI` logging when using their LLM APIs. Their documentation shows that they use `OPENAI_LOG` environment variale to control their loggers. Other LLM vendors will work similarly.

In [32]:
# If you want to log OpenAI's python library itself, also set the log level for this
# normally, limit this to warning/error and keep your own logging at debug levels.
# If this doesn't work right away, restart the kernel after changing the log-level
import os
os.environ["OPENAI_LOG"]="debug"

import openai

# Expects a OPENAI_API_KEY env var
def get_completion(prompt, model="gpt-4o-mini", temperature=0) -> str:
    messages = [{"role":"user", "content":prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature)
    return response.choices[0].message.content

In [4]:
print(get_completion("Why is the sky blue"))

07:43:34 DEBUG:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Why is the sky blue'}], 'model': 'gpt-4o-mini', 'temperature': 0}}
07:43:34 DEBUG:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
07:43:34 DEBUG:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
07:43:34 DEBUG:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f0620a6c050>
07:43:34 DEBUG:start_tls.started ssl_context=<ssl.SSLContext object at 0x7f0620a11d90> server_hostname='api.openai.com' timeout=5.0
07:43:34 DEBUG:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f0620b81590>
07:43:34 DEBUG:send_request_headers.started request=<Request [b'POST']>
07:43:34 DEBUG:send_request_headers.complete
07:43:34 DEBUG:send_request_body.started request=<Request [b'POST']>
07:43:34 DEBUG:send_request_body.complete
0

The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, each with varying wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.

As sunlight passes through the atmosphere, it collides with gas molecules and small particles. Because blue light is scattered in all directions more than other colors due to its shorter wavelength, we see a predominance of blue when we look up at the sky.

During sunrise and sunset, the sky can appear red or orange because the sunlight has to pass through a greater thickness of the atmosphere. This longer path scatters the shorter blue wavelengths out of our line of sight, allowing the longer red wavelengths to dominate the view.


You'll notice two things 👆

 - Those logs 😍
 - Such log! much noise! Where's my actual output at 😟

We'll get to decluttering the visual a bit later.

## Control logging when no env vars are available

> This is useful when you want to change things other than log levels as well. See the formatter example below

It's nice that OpenAI provides the `OPENAL_LOG` env var: very easy to control that. However, in cases where you don't have access to such a variable, you can manipulate the logger directly: you have to get to the logger in use.

### Examine the loggers avaiable.

Note that the loggers are usually initialized at the module level on first use. So you'll likely need to exercise some code to get to them. ALl of this just to get the logger by name. Once you know the name, it usually doesn't change unless some major revision occurs.

In [47]:
import logging
from IPython.display import display, Markdown

# Use this to explore available loggers
# If there is a logger and you are not provided an env-var top control log level, 
# you can directly call logger.setLevel(Logging.DEBUG) to collect logs.
def get_available_loggers():
    return [logging.getLogger(name) for name in logging.root.manager.loggerDict]


# To see them all.
all_loggers = get_available_loggers()
print([l.name for l in all_loggers])

# Say we are interested only in openai
# Long list, I want this formatted nicely. Markdown formatting is easy enough to generate
# compared to HTML
openai_logger_names = [l.name for l in get_available_loggers() if 'openai' in l.name]
display(Markdown(
    "\n".join([f" * {item}" for item in openai_logger_names])
    ))

['httpx', 'rich', 'openai', 'openai._legacy_response', 'openai._response', 'openai._base_client', 'openai.resources.beta.realtime.realtime', 'openai.resources.beta.realtime', 'openai.resources.beta', 'openai.resources', 'openai.audio.transcriptions', 'openai.audio', 'openai.resources.uploads.uploads', 'openai.resources.uploads', 'httpcore.http11', 'httpcore', 'httpcore.connection', 'httpcore.proxy']


 * openai
 * openai._legacy_response
 * openai._response
 * openai._base_client
 * openai.resources.beta.realtime.realtime
 * openai.resources.beta.realtime
 * openai.resources.beta
 * openai.resources
 * openai.audio.transcriptions
 * openai.audio
 * openai.resources.uploads.uploads
 * openai.resources.uploads

### Set the log level directly on the selected logger

The query above shows a logger called `openai`: likely the root logger with individual sub-modules having child loggers. This is how one normally does things so that while testing a sub-module, you can set it's log-level to `INFO` say while reducing the noise down to `ERROR` for everything else.

In [ ]:
# Say we want to customize the 'openai` logger. It likely is inherited by the openai.xxx child-loggers
# but not sure if they copy parent settings on it or will always reference it. Basically, you may have to 
# customize the individual ones or see if root-logger customization carries through.
oai_logger = list(filter(lambda l: l.name == "openai", all_loggers))[0]

# Since we already have all_loggers, I am using a filter on it.
# However, once you know the name, you can also use
# 👉  oai_logger = logging.root.manager.loggerDict.get('openai')
#-----------------------
# Set the level directly
oai_logger.setLevel(logging.DEBUG)


## Distinguishing log output from your cell output 

The main problem _(as illustrated in a previous call to OpenAI's completion API)_ is that of noise. Simply too much stuff and it takes attention away from the output you really care about. Thankfully, there are several easy solutions. The simplest would be to make use of Jupyter notebook's builtin markdown renderer _(also immensely useful when you have LLM output in markdown or want to convert something to markdown for some easy formatting)_. 

> Definitely pays to know your markdown. Mich simpler and less verbose than HTML.

### Use a markdown separator

Simply throw in a markdown separator add/or a markdown section.

In [29]:
from IPython.display import display, Markdown

def markdown_separator(section_name = None):
    if section_name:
        display(Markdown(f"----\n### {section_name}\n"))
    else:
        display(Markdown(f"----"))

In [28]:
# Run to completion so all logs are printed out
res = get_completion("Why is the sky blue")

# print separator
markdown_separator("OpenAI Response")

# print your result
# The use of markdown here formats it into the space available.
# Otw you'll get horizontal scrollbars
display(Markdown(res))

08:00:53 DEBUG:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Why is the sky blue'}], 'model': 'gpt-4o-mini', 'temperature': 0}}
08:00:53 DEBUG:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
08:00:53 DEBUG:close.started
08:00:53 DEBUG:close.complete
08:00:53 DEBUG:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
08:00:53 DEBUG:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f9fb520f460>
08:00:53 DEBUG:start_tls.started ssl_context=<ssl.SSLContext object at 0x7f9fb5256e70> server_hostname='api.openai.com' timeout=5.0
08:00:53 DEBUG:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f9fb520f130>
08:00:53 DEBUG:send_request_headers.started request=<Request [b'POST']>
08:00:53 DEBUG:send_request_headers.complete
08:00:53 DEBUG:send_request_body.started request=<Req

----
# OpenAI Response


The sky appears blue primarily due to a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of different colors, each with varying wavelengths. Blue light has a shorter wavelength compared to other colors like red or yellow.

As sunlight passes through the atmosphere, it collides with gas molecules and small particles. Because blue light is scattered in all directions more than other colors due to its shorter wavelength, we see a predominance of blue when we look up at the sky.

During sunrise and sunset, the sun is lower on the horizon, and its light has to pass through a greater thickness of the atmosphere. This increased distance scatters the shorter blue wavelengths out of our line of sight, allowing the longer wavelengths like red and orange to dominate, which is why the sky can appear red or orange during those times.

### Color the cell output

Take advantage of the `IPython.display.Html` object and render any HTML that you want. 

In [37]:
# For displaying HTML and Markdown responses from ChatGPT
from IPython.display import display, HTML

# Enhance with more Html (fg-color, font, etc) as needed but title is usually a good starting point.
def colorBox(txt, title=None):
    if title is not None:
        txt = f"<b>{title}</b><br><hr><br>{txt}"

    display(HTML(f"<div style='border-radius:15px;padding:15px;background-color:pink;color:black;'>{txt}</div>"))

# Run to completion so all logs are printed out
res = get_completion("Why is the sky blue")

# print your result
# The use of markdown here formats it into the space available.
# Otw you'll get horizontal scrollbars
colorBox(res, title="OpenAI Response")

12:11:31 DEBUG:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'Why is the sky blue'}], 'model': 'gpt-4o-mini', 'temperature': 0}}
12:11:31 DEBUG:Sending HTTP Request: POST https://api.openai.com/v1/chat/completions
12:11:31 DEBUG:close.started
12:11:31 DEBUG:close.complete
12:11:31 DEBUG:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=5.0 socket_options=None
12:11:31 DEBUG:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f9fb4b20710>
12:11:31 DEBUG:start_tls.started ssl_context=<ssl.SSLContext object at 0x7f9fb5256e70> server_hostname='api.openai.com' timeout=5.0
12:11:31 DEBUG:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x7f9fb4ad1440>
12:11:31 DEBUG:send_request_headers.started request=<Request [b'POST']>
12:11:31 DEBUG:send_request_headers.complete
12:11:31 DEBUG:send_request_body.started request=<Req

### Color the log output to quickly zero in on errors

Code below mostly copied from https://stackoverflow.com/questions/68807282/rich-logging-output-in-jupyter-ipython-notebook 🙏

In [ ]:
import logging
from IPython.display import display, HTML

class DisplayHandler(logging.Handler):
    def emit(self, record):
        message = self.format(record)
        display(message)

class HTMLFormatter(logging.Formatter):
    level_colors = {
        logging.DEBUG: 'lightblue',
        logging.INFO: 'dodgerblue',
        logging.WARNING: 'goldenrod',
        logging.ERROR: 'crimson',
        logging.CRITICAL: 'firebrick'
    }
    
    def __init__(self):
        super().__init__(
            '<span style="font-weight: bold; color: green">{asctime}</span> '
            '[<span style="font-weight: bold; color: {levelcolor}">{levelname}</span>] '
            '{message}',
            style='{'
        )
    
    def format(self, record):
        record.levelcolor = self.level_colors.get(record.levelno, 'black')
        return HTML(super().format(record))    

In [ ]:
# One of the cells above releavels a logger called `openai`
# Lets target that and change it's formatter
handler = DisplayHandler()
handler.setFormatter(HTMLFormatter())

oai_logger = logging.root.manager.loggerDict.get('openai')
oai_logger.addHandler(handler)
oai_logger.setLevel(logging.DEBUG)

In [ ]:
# Run to completion so all logs are printed out
res = get_completion("Why is the sky blue")

# print your result
# The use of markdown here formats it into the space available.
# Otw you'll get horizontal scrollbars
colorBox(res, title="OpenAI Response")